# Model Training
## Models to train:
- Classifiers:
  - Identify if observed orientation changes caused by head or phone, spit out class (movement) + cause (head vs phone)
  - Identify gesture, spit-out class (gesture performed, which will account for the cause also, e.g. head-turned-left vs phone-moved-right) - **Probs not this one**
- Regression:
  - Provide a actual value for the relative orientation + position of head and phone (may also need a classifier for the gesture)

## Variations:
- Single Model:</br>
  Take image, output gesture</br>
  Can only be called when image available and prior prediction complete, as such need to average other data (IMU)</br>
  Might not be possible to have *cond* tensor (e.g. change tensor used based on some condition), so may need a multiply based on input, e.g. provide 0 if face not found, and previous image given?
- Split Model:</br>
  Take image, get head pose, feed into second model with motion data to get gesture</br>
  Second model called as and when data (IMU or 1st model output) available (keeping prior values for pending data), first model called as and when image available</br>
  Can also be called with data averaged (e.g. only when image available)</br>
  Possibly use conditional tensor, will use different tensor object based on condition (e.g. face is visible in frame)
- Half Model:</br>
  Use model but only for some processing, e.g. just get landmarks (see: agarwal2020realtime & guobing2021headpose), or to get gesture

Need to worry about whether tools used require region containing face to do their work, and if so how best to obtain region containing face that doesn't care for orientation.


# Loading Data
Preparing the data to use (e.g. read the csv into pandas dataframe)

Need to pre-load the images? Rotate them and resize them?</br>
Probably too large to fit all into memory (can exclude some for testing)

In [ ]:
from random import shuffle
import csv
import cv2 as cv
import numpy as np
import tensorflow as tf
import sklearn as skl
from sklearn.model_selection import train_test_split
from pathlib import Path
import itertools as iter

CLASSES = {
    'NO_GESTURE' : 0,
    'POINTING_TRANSLATE_PHONE_TOP_CENTRE' : 1,
    'POINTING_TRANSLATE_PHONE_TOP_RIGHT' : 2,
    'POINTING_TRANSLATE_PHONE_MID_RIGHT' : 3,
    'POINTING_TRANSLATE_PHONE_BOTTOM_RIGHT' : 4,
    'POINTING_TRANSLATE_PHONE_BOTTOM_CENTRE' : 5,
    'POINTING_TRANSLATE_PHONE_BOTTOM_LEFT' : 6,
    'POINTING_TRANSLATE_PHONE_MID_LEFT' : 7,
    'POINTING_TRANSLATE_PHONE_TOP_LEFT' : 8,
    'POINTING_ROTATE_PHONE_TOP_CENTRE' : 9,
    'POINTING_ROTATE_PHONE_TOP_RIGHT' : 10,
    'POINTING_ROTATE_PHONE_MID_RIGHT' : 11,
    'POINTING_ROTATE_PHONE_BOTTOM_RIGHT' : 12,
    'POINTING_ROTATE_PHONE_BOTTOM_CENTRE' : 13,
    'POINTING_ROTATE_PHONE_BOTTOM_LEFT' : 14,
    'POINTING_ROTATE_PHONE_MID_LEFT' : 15,
    'POINTING_ROTATE_PHONE_TOP_LEFT' : 16,
    'POINTING_ROTATE_HEAD_TOP_CENTRE' : 17,
    'POINTING_ROTATE_HEAD_TOP_RIGHT' : 18,
    'POINTING_ROTATE_HEAD_MID_RIGHT' : 19,
    'POINTING_ROTATE_HEAD_BOTTOM_RIGHT' : 20,
    'POINTING_ROTATE_HEAD_BOTTOM_CENTRE' : 21,
    'POINTING_ROTATE_HEAD_BOTTOM_LEFT' : 22,
    'POINTING_ROTATE_HEAD_MID_LEFT' : 23,
    'POINTING_ROTATE_HEAD_TOP_LEFT' : 24,
    'TRANSLATE_PHONE_TOP_CENTRE': 25,
    'TRANSLATE_PHONE_MID_RIGHT': 26,
    'TRANSLATE_PHONE_BOTTOM_CENTRE': 27,
    'TRANSLATE_PHONE_MID_LEFT': 28,
    'CIRCULAR_PHONE_CLOCKWISE': 29,
    'CIRCULAR_PHONE_ANTI_CLOCKWISE': 30,
    'CIRCULAR_HEAD_CLOCKWISE' : 31,
    'CIRCULAR_HEAD_ANTI_CLOCKWISE' : 32,
    'ZOOM_PHONE_ZOOM_IN' : 33,
    'ZOOM_PHONE_ZOOM_OUT' : 34,
    'ZOOM_HEAD_ZOOM_IN' : 35,
    'ZOOM_HEAD_ZOOM_OUT' : 36,
    'ROTATE_HEAD_TOP_CENTRE' : 37,
    'ROTATE_HEAD_MID_RIGHT' : 38,
    'ROTATE_HEAD_BOTTOM_CENTRE' : 39,
    'ROTATE_HEAD_MID_LEFT' : 40,
    'ROTATE_PHONE_ROLL_CLOCKWISE' : 41,
    'ROTATE_PHONE_ROLL_ANTI_CLOCKWISE : 42'
    'ROTATE_HEAD_ROLL_CLOCKWISE' : 43,
    'ROTATE_HEAD_ROLL_ANTI_CLOCKWISE' : 44
}

OUTPUT_SHAPES = ((1), 1)
OUTPUT_TYPES = (tf.dtypes.float32, tf.dtypes.uint8)

CSV_FIELD_NAMES = [
    'MS_DELTA', 'RELATIVE_IMAGE_PATH', 'LINEAR_X', 'LINEAR_Y', 'LINEAR_Z', 'ROTATION_X', 'ROTATION_Y', 'ROTATION_Z', 'ROTATION_SCALAR', 
    'LINEAR_X_DELTA', 'LINEAR_Y_DELTA', 'LINEAR_Z_DELTA', 'ROTATION_X_DELTA', 'ROTATION_Y_DELTA', 'ROTATION_Z_DELTA', 'ROTATION_SCALAR_DELTA'
]
# CSV_CLASSIFIER_FIELD_NAMES = []
CSV_REGRESSION_FIELD_NAMES = [
    'HEAD_X', 'HEAD_Y', 'HEAD_Z', 'HEAD_PITCH', 'HEAD_ROLL', 'HEAD_YAW', 'PHONE_X', 'PHONE_Y', 'PHONE_Z', 'PHONE_PITCH', 'PHONE_ROLL', 'PHONE_YAW', 
    'HEAD_X_DELTA', 'HEAD_Y_DELTA', 'HEAD_Z_DELTA', 'HEAD_PITCH_DELTA', 'HEAD_ROLL_DELTA', 'HEAD_YAW_DELTA', 
    'PHONE_X_DELTA', 'PHONE_Y_DELTA', 'PHONE_Z_DELTA', 'PHONE_PITCH_DELTA', 'PHONE_ROLL_DELTA', 'PHONE_YAW_DELTA'
]

def get_preprocessed_image(image_path):
    cv.imread()
    pass

# Need to batch (chunk size is hyper-param)
# probs want to over-crank to get cases where have less frames (e.g. keep every x frames, average data between that)
def data_generator(paths, chunk_size=10, pick_rate=[1,2,3,4], regression_output=False):
    config = 
    if average_until_new_image:
        data_paths = shuffle(data_paths + [(path, True) for path in data_paths])
    
    for path in data_paths:
        with open(path, 'r') as csv_file:
            reader = csv.DictReader(csv_file)
            reader.__next__() # skip header
            for row in reader:
                


    

In [ ]:
DATA_PATH = # Replace with path
def get_data_filepaths(root_dir):
    for participant_path in root_dir.iterdir():
        if participant_path.is_dir():
            synced_data_path = participant_path / 'synced_data'
            for participant_data_path in synced_data_path.iterdir():
                if participant_data_path.is_dir():
                    for gesture in participant_data_path.iterdir():
                        for direction in gesture.iterdir():
                            if len(list((direction / 'images').iterdir())) > 0:
                                csv = synced_data_path / f'{participant_data_path.name}_{gesture.name}_{direction.name}.csv'
                                if Path(csv).is_file():
                                    yield csv

file_paths = list(get_data_filepaths(DATA_PATH))
print(f'Number of files to process: {len(file_paths)}')

train_val_paths, test_paths = train_test_split(file_paths, test_size=0.2, random_state=28)
train_paths, val_paths = train_test_split(train_val_paths, test_size=0.3, random_state=16)

print(f"Testing paths: {len(test_paths)}, Training: {len(train_paths)}, Validation: {len(val_paths)}")

train_ds = tf.data.Dataset.from_generator(data_generator(train_paths))
val_ds = tf.data.Dataset.from_generator(data_generator(val_paths))
test_ds = tf.data.Dataset.from_generator(data_generator(test_paths))